In [57]:
#Question 1 Solution

import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import Row
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql.window import Window

spark = SparkSession.builder.appName('abc').getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

#Reading the csv file(I could have read the same file from hdfs, but it wont work in this setup, so im reading file)
data =spark.read.csv("\\Users\\PoonamYadav\\Downloads\\lending-club-loan-data\\new_sample_loan.csv",header=True)

#Registering table as Dataframe
table = sqlContext.registerDataFrameAsTable(data, "Loan_table")

#Selecting good_loan as‘Current’, ‘Issued’ and ‘Fully Paid’ 
good_loan= spark.sql("SELECT term,loan_status as good_loan, ROW_NUMBER() OVER (PARTITION BY term ORDER BY term)as rn from Loan_table WHERE (loan_status = 'Current' or loan_status = 'Fully Paid' or loan_status = 'Issued')")



rs =good_loan.groupby("term").count().show()




+----------+-----+
|      term|count|
+----------+-----+
| 36 months|   45|
| 60 months|   11|
+----------+-----+



In [ ]:
#Solution of Question two

In [54]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions


# Create a SparkSession (the config bit is only for Windows!)
spark = SparkSession.builder.config("spark.sql.warehouse.dir", "file:///C:/temp").appName("Loan_data").getOrCreate()


# Get the raw data
lines = spark.sparkContext.textFile("\\Users\\PoonamYadav\\Downloads\\lending-club-loan-data\\sample_loan.csv")

# Convert it to a RDD of Row objects
Emp = lines.map(lambda x: (x.split(",")[5]))

# Map it it x,1
Empdata = Emp.map(lambda x: (x,1))
emp_by_key = Empdata.reduceByKey(lambda x, y: x+y)

#Finding the max emp_title
max_sortByKey = emp_by_key.map(lambda x: (x[1],x[0])).sortByKey(ascending= False)
max_results = max_sortByKey.first()
print("Employee who took most loan is : {}".format(max_results))

#Finding the least emp_title
min_sortByKey = emp_by_key.sortByKey()
min_results = min_sortByKey.first()
print("Employee who took least loan is : {}".format(min_results))


Employee who is most loan is : (4, 'Starbucks')
Employee who is least loan is : ('"Gen-Probe', 1)


In [1]:
#Solution of question3 
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import Row
from pyspark import SparkContext
from pyspark.conf import SparkConf

spark = SparkSession.builder.appName('abc').getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

def mapper(line):
    fields = line.split(',')
    return Row(loan_status=str(fields[2]), purpose=str(fields[3]))

lines = sc.textFile("\\Users\\PoonamYadav\\Downloads\\lending-club-loan-data\\new_sample_loan.csv")
Loan_table = lines.map(mapper)
#Registering table as Dataframe
schemaPeople = spark.createDataFrame(Loan_table).cache()
schemaPeople.createOrReplaceTempView("Loan_table")


#Selecting good_loan as‘Current’, ‘Issued’ and ‘Fully Paid’ 
bad_loan= spark.sql("SELECT loan_status, purpose from Loan_table WHERE ((loan_status <> 'Current')  and (loan_status <> 'Fully Paid') and (loan_status <> 'Issued'))")

bad_loan.show()


+------------+------------------+
| loan_status|           purpose|
+------------+------------------+
| loan_status|           purpose|
| Charged Off|               car|
| Charged Off|    small_business|
| Charged Off|             other|
| Charged Off|debt_consolidation|
| Charged Off|             other|
| Charged Off|debt_consolidation|
| Charged Off|    major_purchase|
| Charged Off|debt_consolidation|
| Charged Off|             other|
|       Inc."|        Fully Paid|
| Charged Off|debt_consolidation|
| Charged Off|debt_consolidation|
| Charged Off|               car|
| Charged Off|debt_consolidation|
| Charged Off|debt_consolidation|
| Charged Off|debt_consolidation|
|        Inc"|        Fully Paid|
| Charged Off|    small_business|
| California"|       Charged Off|
+------------+------------------+



In [18]:
#Solution of question3 Continued

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import Row
from pyspark import SparkContext
from pyspark.conf import SparkConf

spark = SparkSession.builder.appName('abc').getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

def mapper(line):
    fields = line.split(',')
    return Row(purpose=str(fields[3]))

lines = sc.textFile("\\Users\\PoonamYadav\\Downloads\\lending-club-loan-data\\new_sample_loan.csv")
Purpose_table = lines.map(mapper)
#Registering table as Dataframe
schemaPeople = spark.createDataFrame(Purpose_table).cache()
schemaPeople.createOrReplaceTempView("Purpose_table")
purpose_loan =spark.sql("SELECT purpose,count(*) as cnt from Purpose_table GROUP BY purpose")
purpose_loan.show()
new_loan = bad_loan.join(purpose_loan, bad_loan.purpose == purpose_loan.purpose).select(bad_loan.purpose, purpose_loan.cnt)
new_loan.show()
new_loan.orderBy("cnt", ascending=False).first()




+------------------+---+
|           purpose|cnt|
+------------------+---+
|        Fully Paid|  2|
|           purpose|  1|
|           wedding|  1|
|             other|  8|
|    small_business|  5|
|       Charged Off|  1|
|debt_consolidation| 34|
|       credit_card| 14|
|            moving|  1|
|               car|  3|
|    major_purchase|  2|
|           medical|  1|
|  home_improvement|  1|
+------------------+---+

+------------------+---+
|           purpose|cnt|
+------------------+---+
|           purpose|  1|
|               car|  3|
|    small_business|  5|
|             other|  8|
|debt_consolidation| 34|
|             other|  8|
|debt_consolidation| 34|
|    major_purchase|  2|
|debt_consolidation| 34|
|             other|  8|
|        Fully Paid|  2|
|debt_consolidation| 34|
|debt_consolidation| 34|
|               car|  3|
|debt_consolidation| 34|
|debt_consolidation| 34|
|debt_consolidation| 34|
|        Fully Paid|  2|
|    small_business|  5|
|       Charged Off|  1|

Row(purpose='debt_consolidation', cnt=34)